In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1106899,2021-03-21,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1106900,2021-03-22,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1106901,2021-03-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1106902,2021-03-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
29827,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29829,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29831,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29833,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29835,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
683705,2021-03-21,Arkansas,Arkansas,5001,2024,35.00,5000,Arkansas,AR,Arkansas,State,3017804
683707,2021-03-22,Arkansas,Arkansas,5001,2024,35.00,5000,Arkansas,AR,Arkansas,State,3017804
683709,2021-03-23,Arkansas,Arkansas,5001,2027,35.00,5000,Arkansas,AR,Arkansas,State,3017804
683711,2021-03-24,Arkansas,Arkansas,5001,2031,35.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1106899,2021-03-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1106900,2021-03-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1106901,2021-03-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1106902,2021-03-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-03-23') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
427,2021-03-23,Snohomish,Washington,32195,554.00,53061,WA,County,822083,3916.27,67.39
854,2021-03-23,Cook,Illinois,489484,10153.00,17031,IL,County,5150233,9504.11,197.14
1280,2021-03-23,Orange,California,264989,4635.00,6059,CA,County,3175692,8344.29,145.95
1705,2021-03-23,Maricopa,Arizona,521908,9571.00,4013,AZ,County,4485414,11635.67,213.38
2130,2021-03-23,Los Angeles,California,1215776,22871.00,6037,CA,County,10039107,12110.40,227.82
...,...,...,...,...,...,...,...,...,...,...,...
1106369,2021-03-23,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08
1106533,2021-03-23,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1106666,2021-03-23,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1106795,2021-03-23,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
427,2021-03-23,Snohomish,Washington,32195,554.00,53061,WA,County,822083,3916.27,67.39,67.39,3916.27
854,2021-03-23,Cook,Illinois,489484,10153.00,17031,IL,County,5150233,9504.11,197.14,197.14,9504.11
1280,2021-03-23,Orange,California,264989,4635.00,6059,CA,County,3175692,8344.29,145.95,145.95,8344.29
1705,2021-03-23,Maricopa,Arizona,521908,9571.00,4013,AZ,County,4485414,11635.67,213.38,213.38,11635.67
2130,2021-03-23,Los Angeles,California,1215776,22871.00,6037,CA,County,10039107,12110.40,227.82,227.82,12110.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106369,2021-03-23,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08,75.08,1876.88
1106533,2021-03-23,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1106666,2021-03-23,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1106795,2021-03-23,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
777935,2021-03-23,Clarke,Alabama,3478,57.00,1025,AL,County,23622,14723.56,241.30,241.30,14723.56,1
714840,2021-03-23,Hale,Alabama,2149,72.00,1065,AL,County,14651,14667.94,491.43,491.43,14667.94,2
559579,2021-03-23,Lowndes,Alabama,1357,52.00,1085,AL,County,9726,13952.29,534.65,534.65,13952.29,3
416554,2021-03-23,Franklin,Alabama,4180,82.00,1059,AL,County,31362,13328.23,261.46,261.46,13328.23,4
496267,2021-03-23,Etowah,Alabama,13608,339.00,1055,AL,County,102268,13306.22,331.48,331.48,13306.22,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682262,2021-03-23,Converse,Wyoming,981,17.00,56009,WY,County,13822,7097.38,122.99,122.99,7097.38,19
682625,2021-03-23,Sublette,Wyoming,689,7.00,56035,WY,County,9831,7008.44,71.20,71.20,7008.44,20
847747,2021-03-23,Lincoln,Wyoming,1316,12.00,56023,WY,County,19830,6636.41,60.51,60.51,6636.41,21
943143,2021-03-23,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
559579,2021-03-23,Lowndes,Alabama,1357,52.00,1085,AL,County,9726,13952.29,534.65,534.65,13952.29,3,1
714840,2021-03-23,Hale,Alabama,2149,72.00,1065,AL,County,14651,14667.94,491.43,491.43,14667.94,2,2
238304,2021-03-23,Walker,Alabama,7017,269.00,1127,AL,County,63521,11046.74,423.48,423.48,11046.74,28,3
558849,2021-03-23,Crenshaw,Alabama,1490,57.00,1041,AL,County,13772,10819.05,413.88,413.88,10819.05,30,4
494803,2021-03-23,Clay,Alabama,1479,54.00,1027,AL,County,13235,11174.91,408.01,408.01,11174.91,23,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601919,2021-03-23,Hot Springs,Wyoming,363,3.00,56017,WY,County,4413,8225.70,67.98,67.98,8225.70,17,19
847747,2021-03-23,Lincoln,Wyoming,1316,12.00,56023,WY,County,19830,6636.41,60.51,60.51,6636.41,21,20
826684,2021-03-23,Uinta,Wyoming,2111,12.00,56041,WY,County,20226,10437.06,59.33,59.33,10437.06,4,21
237188,2021-03-23,Teton,Wyoming,3532,9.00,56039,WY,County,23464,15052.85,38.36,38.36,15052.85,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.39,3916.27,15,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.39,3916.27,15,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.39,3916.27,15,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.39,3916.27,15,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.39,3916.27,15,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104295,2021-03-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1104296,2021-03-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1104297,2021-03-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1104298,2021-03-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
774975,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,241.30,14723.56,35,1,2.00
774976,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,241.30,14723.56,35,1,0.00
774977,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,241.30,14723.56,35,1,1.00
774978,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,241.30,14723.56,35,1,4.00
774979,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,241.30,14723.56,35,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861873,2021-03-21,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00
861874,2021-03-22,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00
861875,2021-03-23,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00
861876,2021-03-24,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
557339,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13952.29,1,3,1.00,0.00
557340,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13952.29,1,3,0.00,0.00
557341,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13952.29,1,3,0.00,0.00
557342,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13952.29,1,3,0.00,0.00
557343,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13952.29,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555509,2021-03-21,Albany,Wyoming,3936,11.00,56001,WY,County,38880,10123.46,28.29,28.29,10162.04,23,7,0.00,0.00
555510,2021-03-22,Albany,Wyoming,3947,11.00,56001,WY,County,38880,10151.75,28.29,28.29,10162.04,23,7,11.00,0.00
555511,2021-03-23,Albany,Wyoming,3951,11.00,56001,WY,County,38880,10162.04,28.29,28.29,10162.04,23,7,4.00,0.00
555512,2021-03-24,Albany,Wyoming,3952,11.00,56001,WY,County,38880,10164.61,28.29,28.29,10162.04,23,7,1.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-03-23') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
284400,2021-03-23,Imperial,California,27302,711.00,6025,CA,County,181215,15066.08,392.35,392.35,15066.08,1,2,35.00,13.00
2130,2021-03-23,Los Angeles,California,1215776,22871.00,6037,CA,County,10039107,12110.40,227.82,227.82,12110.40,2,5,446.00,37.00
497683,2021-03-23,Inyo,California,1363,35.00,6027,CA,County,18039,7555.85,194.02,194.02,7555.85,3,24,0.00,0.00
141754,2021-03-23,San Bernardino,California,289892,4107.00,6071,CA,County,2180085,13297.28,188.39,188.39,13297.28,4,4,46.00,13.00
63455,2021-03-23,Stanislaus,California,58347,982.00,6099,CA,County,550660,10595.83,178.33,178.33,10595.83,5,9,73.00,2.00
78652,2021-03-23,Tulare,California,48878,807.00,6107,CA,County,466195,10484.45,173.10,173.10,10484.45,6,10,46.00,0.00
29440,2021-03-23,Riverside,California,293436,4164.00,6065,CA,County,2470546,11877.37,168.55,168.55,11877.37,7,6,287.00,23.00
50503,2021-03-23,San Joaquin,California,69112,1231.00,6077,CA,County,762148,9068.06,161.52,161.52,9068.06,8,19,257.00,0.00
603318,2021-03-23,Kings,California,22532,239.00,6031,CA,County,152940,14732.57,156.27,156.27,14732.57,9,3,8.00,1.00
28672,2021-03-23,Fresno,California,98276,1556.00,6019,CA,County,999101,9836.44,155.74,155.74,9836.44,10,14,65.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1050629,2021-03-23,Lassen,California,5637,1.00,6035,CA,County,30573,18437.84,3.27,3.27,18437.84,56,1,0.00,0.00
284400,2021-03-23,Imperial,California,27302,711.00,6025,CA,County,181215,15066.08,392.35,392.35,15066.08,1,2,35.00,13.00
603318,2021-03-23,Kings,California,22532,239.00,6031,CA,County,152940,14732.57,156.27,156.27,14732.57,9,3,8.00,1.00
141754,2021-03-23,San Bernardino,California,289892,4107.00,6071,CA,County,2180085,13297.28,188.39,188.39,13297.28,4,4,46.00,13.00
2130,2021-03-23,Los Angeles,California,1215776,22871.00,6037,CA,County,10039107,12110.40,227.82,227.82,12110.40,2,5,446.00,37.00
29440,2021-03-23,Riverside,California,293436,4164.00,6065,CA,County,2470546,11877.37,168.55,168.55,11877.37,7,6,287.00,23.00
178171,2021-03-23,Kern,California,105625,1178.00,6029,CA,County,900202,11733.48,130.86,130.86,11733.48,14,7,83.00,8.00
451091,2021-03-23,Merced,California,30298,430.00,6047,CA,County,277680,10911.12,154.85,154.85,10911.12,11,8,41.00,1.00
63455,2021-03-23,Stanislaus,California,58347,982.00,6099,CA,County,550660,10595.83,178.33,178.33,10595.83,5,9,73.00,2.00
78652,2021-03-23,Tulare,California,48878,807.00,6107,CA,County,466195,10484.45,173.10,173.10,10484.45,6,10,46.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
284400,2021-03-23,Imperial,California,27302,711.00,6025,CA,County,181215,15066.08,392.35,392.35,15066.08,1,2,35.00,13.00
2130,2021-03-23,Los Angeles,California,1215776,22871.00,6037,CA,County,10039107,12110.40,227.82,227.82,12110.40,2,5,446.00,37.00
497683,2021-03-23,Inyo,California,1363,35.00,6027,CA,County,18039,7555.85,194.02,194.02,7555.85,3,24,0.00,0.00
141754,2021-03-23,San Bernardino,California,289892,4107.00,6071,CA,County,2180085,13297.28,188.39,188.39,13297.28,4,4,46.00,13.00
63455,2021-03-23,Stanislaus,California,58347,982.00,6099,CA,County,550660,10595.83,178.33,178.33,10595.83,5,9,73.00,2.00
78652,2021-03-23,Tulare,California,48878,807.00,6107,CA,County,466195,10484.45,173.10,173.10,10484.45,6,10,46.00,0.00
29440,2021-03-23,Riverside,California,293436,4164.00,6065,CA,County,2470546,11877.37,168.55,168.55,11877.37,7,6,287.00,23.00
50503,2021-03-23,San Joaquin,California,69112,1231.00,6077,CA,County,762148,9068.06,161.52,161.52,9068.06,8,19,257.00,0.00
603318,2021-03-23,Kings,California,22532,239.00,6031,CA,County,152940,14732.57,156.27,156.27,14732.57,9,3,8.00,1.00
28672,2021-03-23,Fresno,California,98276,1556.00,6019,CA,County,999101,9836.44,155.74,155.74,9836.44,10,14,65.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.35,15066.08,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.35,15066.08,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.35,15066.08,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.35,15066.08,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.35,15066.08,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3722,03/21/21,Lassen,5634,1.00,18428.02,3.27,3.27,18437.84,56,1,0.00,0.00
3723,03/22/21,Lassen,5637,1.00,18437.84,3.27,3.27,18437.84,56,1,3.00,0.00
3724,03/23/21,Lassen,5637,1.00,18437.84,3.27,3.27,18437.84,56,1,0.00,0.00
3725,03/24/21,Lassen,5637,1.00,18437.84,3.27,3.27,18437.84,56,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.35,15066.08,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.35,15066.08,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.35,15066.08,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.35,15066.08,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.35,15066.08,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3805,03/21/21,Fresno,98112,1532.00,9820.03,153.34,155.74,9836.44,10,14,143.00,5.00
3806,03/22/21,Fresno,98211,1556.00,9829.94,155.74,155.74,9836.44,10,14,99.00,24.00
3807,03/23/21,Fresno,98276,1556.00,9836.44,155.74,155.74,9836.44,10,14,65.00,0.00
3808,03/24/21,Fresno,98362,1575.00,9845.05,157.64,155.74,9836.44,10,14,86.00,19.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)